In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    '''
    Displays a list of frames as a gif, with controls
    '''
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    anim.save('movie_cartpole2.mp4') 
    display(display_animation(anim, default_mode='loop'))

In [2]:
# 定数の設定
ENV = 'CartPole-v0'  # 使用する課題名
NUM_DIZITIZED = 6  # 各状態の離散値への分割数
GAMMA = 0.99  # 時間割引率
ETA = 0.5  # 学習係数
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 1000  # 最大試行回数

In [3]:
class Agent:
    
    
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states,num_actions)
        #智能体的大脑做出决策
        
    def update_Q_func(self, obserbation, action, reward, observation_next):
        self.brain.update_Q_table(obserbation, action, reward, observation_next)
        
    def get_action(self, observation, step):
        action  = self.brain.decide_action(observation, step)
        return action


In [10]:
class Brain:
    
    
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        #用state大小创建Q Table
        #先随机一个均匀分布
        self.q_table = np.random.uniform(low=0,high=1,size=(NUM_DIZITIZED**num_states,num_actions))
        
    #离散化
    def bins(self, clpmin,clpmax,num):
        return np.linspace(clpmin,clpmax,num+1)[1:-1]
    
    def digitize_state(self, observation):
        pos, v, angle, w = observation
        digitized = [
            np.digitize(pos,bins=self.bins(-2.4,2.4,NUM_DIZITIZED)),
            np.digitize(v,bins=self.bins(-3.0,3.0,NUM_DIZITIZED)),
            np.digitize(angle,bins=self.bins(-0.5,0.5,NUM_DIZITIZED)),
            np.digitize(w,bins=self.bins(-2.0,2.0,NUM_DIZITIZED))
        ]

        return sum([x * (NUM_DIZITIZED**i) for i,x in enumerate(digitized)])
    
    def update_Q_table(self,observation, action, reward, observation_next):
        state = self.digitize_state(observation)
        state_next = self.digitize_state(observation_next)
        Max_Q_next = max(self.q_table[state_next][:])
        self.q_table[state, action] = self.q_table[state, action]+ \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action]) #TD过程更新Q
        
    def decide_action(self, observation, epsilon):
        state = self.digitize_state(observation)
        epsilon = 0.5 * (1/(epsilon + 1))
        
        if epsilon <= np.random.uniform(0,1):
            action = np.argmax(self.q_table[state][:])
        else:
            action = np.random.choice(self.num_actions)
        
        return action
    

In [11]:
class Environment:
    
    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態の数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成
        
    def run(self):
        
        complete_episodes = 0
        is_episode_final = False
        frames = []
        
        for episode in range(NUM_EPISODES):
            observation = self.env.reset()
            
            for step in range(MAX_STEPS):
                
                if is_episode_final is True:
                    
                    frames.append(self.env.render(mode='rgb_array'))
                    
                action = self.agent.get_action(observation, episode)
                
                observation_next, _, done, _=self.env.step(action)
                
                if done:
                    if step<195:
                        reward = -1
                        complete_episodes=0
                    else:
                        reward = 1
                        complete_episodes+=1
                else:
                    reward = 0
                
                self.agent.update_Q_func(observation,action,reward,observation_next)
                
                observation = observation_next
                
                if done:
                    print('{0} 轮实验： 用了{1}步'.format(episode, step+1))
                    break
                    
                if complete_episodes>=10:
                    print("10回合连续成功！！！！\n")
                    is_episode_final = True
                
            

In [ ]:
cartpole_env = Environment()
cartpole_env.run()

D:\anaconda3\envs\rlenv\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
0 轮实验： 用了25步
1 轮实验： 用了22步
2 轮实验： 用了20步
3 轮实验： 用了42步
4 轮实验： 用了18步
5 轮实验： 用了84步
6 轮实验： 用了16步
7 轮实验： 用了15步
8 轮实验： 用了14步
9 轮实验： 用了14步
10 轮实验： 用了28步
11 轮实验： 用了19步
12 轮实验： 用了44步
13 轮实验： 用了99步
14 轮实验： 用了139步
15 轮实验： 用了68步
16 轮实验： 用了34步
17 轮实验： 用了107步
18 轮实验： 用了46步
19 轮实验： 用了99步
20 轮实验： 用了66步
21 轮实验： 用了86步
22 轮实验： 用了55步
23 轮实验： 用了83步
24 轮实验： 用了40步
25 轮实验： 用了69步
26 轮实验： 用了21步
27 轮实验： 用了48步
28 轮实验： 用了11步
29 轮实验： 用了14步
30 轮实验： 用了9步
31 轮实验： 用了24步
32 轮实验： 用了22步
33 轮实验： 用了26步
34 轮实验： 用了35步
35 轮实验： 用了199步
36 轮实验： 用了26步
37 轮实验： 用了9步
38 轮实验： 用了118步
39 轮实验： 用了73步
40 轮实验： 用了138步
41 轮实验： 用了108步
42 轮实验： 用了103步
43 轮实验： 用了39步
44 轮实验： 用了46步
45 轮实验： 用了37步
46 轮实验： 用了50步
47 轮实验： 用了19步
48 轮实验： 用了24步
49 轮实验： 用了170步
50 轮实验： 用了179步
51 轮实验： 用了56步
52 轮实验： 用了95步
53 轮实验： 用了138步
54 轮实验： 用了15步
55 轮实验： 用了18步
56 轮实验： 用了23步
57 轮实验： 用了18步
58 轮实验： 用了23步
59 轮实验： 用了199步
60 轮实验： 用了182步
61 轮实验： 用了20步
62 轮实验： 用了10步
63 轮实验： 用了133步
64 轮

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续

...

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

212 轮实验： 用了200步
10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

215 轮实验： 用了200步
10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

218 轮实验： 用了200步
10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

221 轮实验： 用了200步
10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

10回合连续成功！！！！

